In [1]:
import os
import sys
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation

from torch_wavefunction import *
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

Using device: cuda


In [2]:
class WavefunctionResults:
    def __init__(self):
        self.wavefunctions = []
        self.convergence_energies = []
        self.final_wavefunctions = []
        self.final_energies = []
        self.torch_wavefunctions = []

In [3]:
def train_single_state(results: WavefunctionResults, x_train,layout=[1,60,60,1], **kwargs):
    model = WaveFunctionMLP(layout).to(device)
    
    if "callback" in kwargs:
        callback_kwargs = kwargs["callback"]
        assert isinstance(callback_kwargs, dict), "callback_kwargs must be a dictionary"
    else:
        callback_kwargs = {}
    if "train" in kwargs:
        train_kwargs = kwargs["train"]
        assert isinstance(train_kwargs, dict), "train_kwargs must be a dictionary"
    else:
        train_kwargs = {}
    
    
    for key in kwargs:
        if ((key != "callback") and (key != "train")):
            print(f"Unknown keyword argument: {key}")
    
    if "patience" not in callback_kwargs:
        callback_kwargs['patience'] = 250
    if "min_delta" not in callback_kwargs:
        callback_kwargs['min_delta'] = 0.0001
        
    if "lr" not in train_kwargs:
        train_kwargs['lr'] = 0.001
    if "epochs" not in train_kwargs:
        train_kwargs['epochs'] = 1000
    if "save_wavefunction_history" not in train_kwargs:
        train_kwargs['save_wavefunction_history'] = False
    # if "save_energy_history" not in train_kwargs:
    #     train_kwargs['save_energy_history'] = False
    if "previous_wavefunctions" not in train_kwargs:
        train_kwargs['previous_wavefunctions'] = None
    
    early_stopping = EarlyStoppingCallback(patience = callback_kwargs['patience'], min_delta =  callback_kwargs['min_delta'])
    psi_normalized_cpu, energy_fin, energy_hist, wf_hist, psi_normalized_torch = train_wavefunction(model, 
                                                                                                    x_train, 
                                                                                                    callback=early_stopping, 
                                                                                                    **train_kwargs)
    results.wavefunctions.append(wf_hist)
    results.convergence_energies.append(energy_hist)
    results.final_wavefunctions.append(psi_normalized_cpu)
    results.final_energies.append(energy_fin)
    results.torch_wavefunctions.append(psi_normalized_torch)
    return True

The data in the model has to have the shape `[N_batch,N_input]`, where `N_batch` is the batch size (or in our case the number of points in space), and `N_input` is the number of input neurons.

In [5]:
x_dim = torch.linspace(-10, 10, 10, dtype=torch.float32, device=device, requires_grad=True).view(-1, 1)
y_dim = torch.linspace(-10, 10, 10, dtype=torch.float32, device=device, requires_grad=True).view(-1, 1)
x_train = torch.stack([x_dim, y_dim], dim=1).to(device)

# transpose x_Train
x_train = (torch.squeeze(x_train)).detach()
x_train.requires_grad = True
x_train.shape

torch.Size([10, 2])

In [6]:

results = WavefunctionResults()

callback_kwargs_0 = {
    "patience": 250,
    "min_delta": 1e-5,
    }
train_kwargs_0 = {
    "lr": 0.001,
    "epochs": 1000,
    "save_wavefunction_history": True,
    # "save_energy_history": True,
    "previous_wavefunctions": None,
    }

options_0 = {
    "callback": callback_kwargs_0,
    "train": train_kwargs_0,
    }

train_single_state(results, x_train, layout=[2,60,60,1], **options_0)

/home/pfargas/software/anaconda3/lib/python3.12/site-packages/torch/autograd/graph.py:824: UserWarning: Attempting to run cuBLAS, but there was no current CUDA context! Attempting to set the primary context... (Triggered internally at /pytorch/aten/src/ATen/cuda/CublasHandlePool.cpp:181.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


RuntimeError: grad can be implicitly created only for scalar outputs

In [11]:
class WaveFunctionMLP(nn.Module):
    
    def __init__(self, layer_dims=[1,60,60,1]):
        super(WaveFunctionMLP, self).__init__()
        layers = []
        for i in range(len(layer_dims) - 1):
            layers.append(nn.Linear(layer_dims[i], layer_dims[i+1]))
            if i < len(layer_dims) - 2:  # Adding activation function for all layers except the last lasyer
                layers.append(nn.Tanh())

        self.net = nn.Sequential(*layers)

        
    def forward(self, x):
        return self.net(x)

In [12]:
model = WaveFunctionMLP([2,1,1]).to(device)

In [41]:
test_argument_noT = torch.tensor([[1.0, 2.0, 5.0],[3.0, 4.0, 6.0]], device=device)
test_argument = test_argument_noT.transpose(0, 1)
print(test_argument.shape)

first_degree = torch.tensor([1.0,2.0, 3.0], device=device)
second_degree = torch.tensor([4.0,5.0, 6.0], device=device)

print(first_degree.shape)

first_degree = torch.linspace(1, 2, 10, dtype=torch.float32, device=device).view(-1, 1)
second_degree = torch.linspace(3, 4, 10, dtype=torch.float32, device=device).view(-1, 1)



test_arg_combined = torch.stack([first_degree, second_degree], dim=0)

print(test_arg_combined.shape)

test_arg_combined = (torch.squeeze(test_arg_combined)).transpose(0,1)

print(test_argument.shape)
print(test_arg_combined.shape)
output = model(test_arg_combined)
print(output)

torch.Size([3, 2])
torch.Size([3])
torch.Size([2, 10, 1])
torch.Size([3, 2])
torch.Size([10, 2])
tensor([[-0.0302],
        [-0.0313],
        [-0.0321],
        [-0.0328],
        [-0.0333],
        [-0.0337],
        [-0.0340],
        [-0.0342],
        [-0.0344],
        [-0.0345]], device='cuda:0', grad_fn=<AddmmBackward0>)
